In [1]:
import scrape
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
os.system("pkill -f -9 chromium")

%load_ext autoreload
%autoreload 2

In [2]:
class Parent:
    def twice(self):
        self.message()
        self.message()
        
    def message(self):
        print("parent says hi")
        
class Child(Parent):
    def message(self):
        print("child says hi")
        
c = Child()

In [3]:
c.twice()

child says hi
child says hi


In [4]:
class GraphSearcher:
    def __init__(self):
        self.visited = set()
        self.order = []

    def visit_and_get_children(self, node):
        """ 
        Leave this method as is! It will be over-written the child classes
        Each child class should perform the following:
            Record the node value in self.order AND return its children
            parameter: node
            return: children of the given node
        """
        raise Exception("must be overridden in sub classes -- don't change me here!")

    def dfs_search(self, node):
        # 1. clear out visited set and order list
        self.visited.clear()
        self.order.clear()
        # 2. start recursive search by calling dfs_visit
        return self.dfs_visit(node)
        
    def dfs_visit(self, node):
        # 1. if this node has already been visited, just `return` (no value necessary)
        if node in self.visited:
            return
        # 2. mark node as visited by adding it to the set
        self.visited.add(node)
        # 3. call self.visit_and_get_children(node) to get the children
        for child in self.visit_and_get_children(node):
            self.dfs_visit(child)
        # 4. in a loop, call dfs_visit on each of the children
        

In [5]:
g = GraphSearcher()
g.dfs_search("A")

Exception: must be overridden in sub classes -- don't change me here!

In [ ]:
import pandas as pd

df = pd.DataFrame([
    [0,1,0,1],
    [0,0,1,0],
    [0,0,0,1],
    [0,0,1,0],
], index=["A", "B", "C", "D"], columns=["A", "B", "C", "D"])
df

In [ ]:
for node, has_edge in df.loc["B"].items():
    if has_edge == 1:
        print(node)

In [ ]:
class MatrixSearcher(GraphSearcher):
    def __init__(self, df):
        super().__init__() # call constructor method of parent class
        self.df = df
        
    def visit_and_get_children(self, node):
        """ 
        Leave this method as is! It will be over-written the child classes
        Each child class should perform the following:
            Record the node value in self.order AND return its children
            parameter: node
            return: children of the given node
        """
        # TODO: Record the node value in self.order
        self.order.append(node)
        children = []
        # TODO: use `self.df` to determine what children the node has and append them
        for node, has_edge in df.loc[node].items():
            if has_edge == 1:
                children.append(node)
        return children

In [ ]:
m = MatrixSearcher(df)
m.dfs_search('A')
m.order

In [ ]:
df = pd.DataFrame([
    [0,1,0,1],
    [0,0,1,0],
    [0,0,0,1],
    [0,0,1,0],
], index=["A", "B", "C", "D"], columns=["A", "B", "C", "D"])

m = scrape.MatrixSearcher(df)
m.bfs_search("A")
m.order

In [ ]:
fs = scrape.FileSearcher()

In [ ]:
fs.visit_and_get_children('1.txt')

In [ ]:
fs = scrape.FileSearcher()
print(fs.visit_and_get_children("1.txt"), fs.order, fs.concat_order())

In [ ]:
options = Options()
# Run in headless mode (info: https://www.browserstack.com/guide/what-is-headless-browser-testing)
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
ws = scrape.WebSearcher(driver)

In [ ]:
options = Options()
options.add_argument("--headless=new")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)
ws = scrape.WebSearcher(driver)
driver.get("http://34.68.128.6:5000")
driver.page_source

for link in driver.find_elements('tag name','a'):
    if link.get_attribute("href") != None:
        print(link.get_attribute("href"))

In [ ]:
driver.get("http://34.68.128.6:5000/Node_1.html")
for  tr in driver.find_elements('tag name','tr'):
    print(tr.text)

In [ ]:
pd.read_html('http://34.68.128.6:5000/Node_1.html')[0]

In [ ]:
driver = None

def browser():
    global driver

    if not driver:
        os.system("pkill -f -9 chromium")
        options = Options()
        options.add_argument("--headless=new")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)
    return driver

driver = browser()

VM_IP = ''
start_url = f"http://34.68.128.6:5000/Node_1.html"

s = scrape.WebSearcher(driver)
print(s.visit_and_get_children(start_url))

s = scrape.WebSearcher(driver)
s.bfs_search(start_url)

table = s.table()
print(table)

print(''.join(str(x) for x in list(table['clue'])))
driver.close()

In [ ]:
import scrape
options = Options()
# Run in headless mode (info: https://www.browserstack.com/guide/what-is-headless-browser-testing)
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
s = scrape.WebSearcher(driver)
s.bfs_search(start_url)

table = s.table()
scrape.reveal_secrets(driver,"http://34.68.128.6:5000" ,table)

In [ ]:
bfs_points = 0

# test bfs on Matrix
df = pd.DataFrame([
    [0,1,0,1],
    [0,0,1,0],
    [0,0,0,1],
    [0,0,1,0],
], index=["A", "B", "C", "D"], columns=["A", "B", "C", "D"])
m = scrape.MatrixSearcher(df)

m.bfs_search("A")
assert m.order == ['A', 'B', 'D', 'C']
bfs_points += 5

# test visit_and_get_children
f = scrape.FileSearcher()
bfs_points += 2
assert f.visit_and_get_children("1.txt") == ['2.txt', '4.txt']
bfs_points += 3

# test bfs
f = scrape.FileSearcher()
f.bfs_search("1.txt")
msg = f.concat_order()
bfs_points += 5
assert msg == "MADCITY"
bfs_points += 5

# test dfs
f = scrape.FileSearcher()
f.dfs_search("1.txt")
msg = f.concat_order()
assert msg == "MACTIDY"
bfs_points += 5

In [ ]:
f = scrape.FileSearcher()
f.visit_and_get_children("1.txt")

In [ ]:
f = scrape.FileSearcher()
f.bfs_search("1.txt")
msg = f.concat_order()
msg

In [ ]:
web_points = 0
port = "5001"

ws = scrape.WebSearcher(browser())
url = f"http://localhost:{port}/Node_1.html"
assert ws.visit_and_get_children(url) == [f'http://localhost:{port}/Node_2.html',
                      f'http://localhost:{port}/Node_4.html']
web_points += 5

ws = scrape.WebSearcher(browser())

if search_alg == "bfs":
    ws.bfs_search(url)
    assert ws.order == [f'http://localhost:{port}/Node_1.html',
                        f'http://localhost:{port}/Node_2.html',
                        f'http://localhost:{port}/Node_4.html',
                        f'http://localhost:{port}/Node_3.html',
                        f'http://localhost:{port}/Node_5.html',
                        f'http://localhost:{port}/Node_6.html',
                        f'http://localhost:{port}/Node_7.html']
    web_points += 5
else:
    ws.dfs_search(url)
    assert ws.order == [f'http://localhost:{port}/Node_1.html',
                        f'http://localhost:{port}/Node_2.html',
                        f'http://localhost:{port}/Node_3.html',
                        f'http://localhost:{port}/Node_6.html',
                        f'http://localhost:{port}/Node_5.html',
                        f'http://localhost:{port}/Node_4.html',
                        f'http://localhost:{port}/Node_7.html']
    web_points += 5

actual = ws.table()
expected = expected_travellog

# how much overlap was there between the clues?
both = len(set(expected["clue"]).intersection(actual["clue"]))
either = len(set(expected["clue"]).union(actual["clue"]))
web_points += int(both / either * 10)

# did everything match?
assert expected.shape == actual.shape
assert (expected.reset_index() == actual.reset_index()).all().all()
web_points += 5




In [ ]:
web_points = 0
port = "5000"
ws = scrape.WebSearcher(browser())
url = f"http://localhost:{port}/Node_1.html"
ws.visit_and_get_children(url)